In [1]:
import yfinance as yf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from io import StringIO
import re
from datetime import date
#from selenium import webdriver
#from selenium.webdriver.chrome.options import Options
from io import BytesIO
from zipfile import ZipFile
import os
from datetime import datetime, timedelta
from yahooquery import Ticker
import time
import random
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
import warnings
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import norm
from itertools import product

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

warnings.filterwarnings("ignore")


In [2]:
#期权价格及指标实时更新系统
def fetch_nikkei_mini_options():
    """
    抓取日経225ミニオプション（QRI サイト）前 6 個到期月份的数据，
    包含 CALL/PUT 基本报价和对应的希腊字母（Delta, Gamma, Theta, Vega）。
    返回一个 Pandas DataFrame。
    """
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)",
    "Referer":    "https://svc.qri.jp/jpx/nkopmini/"
}
    base_url = "https://svc.qri.jp/jpx/nkopmini/{}"
    records = []

    def to_int(x):
        try: return int(x)
        except: return None

    def to_float(x, pct=False):
        if not x or x in ("-", "--"): return None
        try:
            return float(x.strip('%'))/100 if pct else float(x)
        except:
            return None

    for idx in range(6):
        url = base_url.format(idx)
        resp = requests.get(url, headers=headers)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # 取交易日 & 到期日
        trade_date = pd.to_datetime(
            soup.find("dt", string=re.compile(r"取引日"))
                .find_next_sibling("dd").get_text(strip=True),
            format="%Y/%m/%d"
        )
        expiry_date = pd.to_datetime(
            soup.find("dt", string=re.compile(r"取引最終日"))
                .find_next_sibling("dd").get_text(strip=True),
            format="%Y/%m/%d"
        )

        table = soup.find("table", class_="price-table")
        for row in table.find_all("tr", class_="row-num"):
            cols = [td.get_text(strip=True).replace(",", "") for td in row.find_all("td")]
            m = re.search(r"(\d+)", cols[8])
            strike = float(m.group(1)) if m else None

            # CALL
            call_settlement = to_float(cols[0])
            call_oi         = to_int(cols[1])
            call_volume     = to_int(cols[2])
            call_iv         = to_float(cols[5], pct=True)
            call_mid        = to_float(cols[7])

            # PUT
            put_mid         = to_float(cols[9])
            put_iv          = to_float(cols[12], pct=True)
            put_volume      = to_int(cols[13])
            put_oi          = to_int(cols[14])
            put_settlement  = to_float(cols[15])

            greek_row    = row.find_next_sibling("tr", class_="greek")
            greek_tables = greek_row.find_all("table", class_="greek-value-table")
            call_gks     = [td.get_text(strip=True) for td in greek_tables[0].find_all("td")]
            put_gks      = [td.get_text(strip=True) for td in greek_tables[1].find_all("td")]

            records.append({
                "trade_date":     trade_date,
                "expiry_date":    expiry_date,
                "expiry_index":   idx,
                "strike":         strike,
                # CALL
                "call_settlement": call_settlement,
                "call_oi":         call_oi,
                "call_volume":     call_volume,
                "call_iv":         call_iv,
                "call_mid":        call_mid,
                "call_delta":      to_float(call_gks[0]),
                "call_gamma":      to_float(call_gks[1]),
                "call_theta":      to_float(call_gks[2]),
                "call_vega":       to_float(call_gks[3]),
                # PUT
                "put_settlement":  put_settlement,
                "put_oi":          put_oi,
                "put_volume":      put_volume,
                "put_iv":          put_iv,
                "put_mid":         put_mid,
                "put_delta":       to_float(put_gks[0]),
                "put_gamma":       to_float(put_gks[1]),
                "put_theta":       to_float(put_gks[2]),
                "put_vega":        to_float(put_gks[3]),
            })

    return pd.DataFrame(records)

def update_local_csv(csv_path="nkopmini_chain_with_dates.csv"):
    # 1. 读取已有数据（如果文件不存在则置空）
    if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path, parse_dates=["trade_date", "expiry_date"])
    else:
        df_old = pd.DataFrame()

    # 2. 抓取最新
    df_new = fetch_nikkei_mini_options()

    if df_old.empty:
        df_merged = df_new
    else:
        # 3. 找出新数据中所有不同的 trade_date
        new_dates = df_new["trade_date"].unique()
        # 4. 在旧数据中删除这些日期对应的记录
        df_remaining = df_old[~df_old["trade_date"].isin(new_dates)]
        # 5. 合并：先保留旧的其余，再拼上新的
        df_merged = pd.concat([df_remaining, df_new], ignore_index=True)

    # 6. 按 trade_date, expiry_index, strike 排序并写回 CSV
    df_merged = df_merged.sort_values(
        ["trade_date", "expiry_index", "strike"]
    ).reset_index(drop=True)
    df_merged.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"更新完成，共有 {len(df_merged)} 条记录，最新 trade_date：{df_new['trade_date'].max().date()}")

def compute_theo_ratio(df, underlying_price, r=0.005):
    """
    在 df 中新增：
      - dte: （expiry_date - trade_date）天数
      - call_theo: Black-Scholes 理论价
      - theo_ratio: call_settlement / call_theo
    """
    df = df.copy()
    df['dte'] = (df['expiry_date'] - df['trade_date']).dt.days
    # Black-Scholes 看涨期权
    S = underlying_price
    r = r
    df['ttm'] = df['dte'] / 252
    df['sigma'] = df['call_iv']
    # 防止零波动
    df['sigma'] = df['sigma'].fillna(0.0001)
    df['d1'] = (np.log(S/df['strike']) + (r+0.5*df['sigma']**2)*df['ttm']) / (df['sigma']*np.sqrt(df['ttm']))
    df['d2'] = df['d1'] - df['sigma']*np.sqrt(df['ttm'])
    df['call_theo'] = (S * norm.cdf(df['d1']) - df['strike']*np.exp(-r*df['ttm'])*norm.cdf(df['d2']))
    df['theo_ratio'] = df['call_settlement'] / df['call_theo']
    return df

# 3. 初步过滤
def filter_candidates(df, theo_ratio_min=0.4, dte_limit=40):
    """
    筛选：
      - call_settlement 非空
      - dte ≤ dte_limit
      - theo_ratio ≥ theo_ratio_min
    """
    mask = (
        df['call_settlement'].notna() &
        (df['dte'] <= dte_limit) &
        (df['theo_ratio'] >= theo_ratio_min)
    )
    return df.loc[mask].copy()

# 4. 打分
def score_candidates(df, top_n=None):

    """
    筛选并对候选合约打分：
      - 筛选：20 <= dte <= 40, call_delta∈[0.2, 0.4]
      - 打分指标（正向）：
          * call_delta    权重0.30
          * call_theta    权重0.30
          * call_oi       权重0.15
          * call_volume   权重0.15
          * dte           权重0.10
    返回包含各项子得分及总分的 DataFrame，按 score 降序。
    """
    df = df.copy()

    # 1) 筛选 DTE 和 Delta 范围
    df = df[(df['dte'] >= 5) & (df['dte'] <= 40)]
    df = df[(df['call_delta'] >= 0.2) & (df['call_delta'] <= 0.5)]

    if df.empty:
        return df

    # 2) 归一化并打分
    scaler = MinMaxScaler()
    weights = {
        'call_delta':  0.35,
        'call_theta':  0.35,
        'call_oi':     0.10,
        'call_volume': 0.10,
        'dte':         0.10,
    }

    # 对每个指标做 MinMax 归一化
    for col in weights:
        vals = df[col].fillna(0).values.reshape(-1,1)
        df[f'score_{col}'] = scaler.fit_transform(vals).flatten()

    # 3) 加权求和
    df['score'] = (
        df['score_call_delta'] * weights['call_delta'] +
        df['score_call_theta'] * weights['call_theta'] +
        df['score_call_oi'] * weights['call_oi'] +
        df['score_call_volume'] * weights['call_volume'] +
        df['score_dte'] * weights['dte']
    )

    # 4) 排序并取 top_n
    df = df.sort_values('score', ascending=False).reset_index(drop=True)
    if top_n is not None:
        df = df.head(top_n).copy()

    return df



    print(df_candidates[display_cols])
#发送邮件
def send_df_via_email_html(
    df: pd.DataFrame,
    to_email: str,
    from_email: str,
    app_password: str,
    subject: str = "数据报告",
    preamble: str = "您好，下面是您的数据表格："
):
    # 1) 把 DataFrame 转成 HTML
    html_table = df.to_html(index=False)

    # 2) 构建邮件
    msg = MIMEMultipart("alternative")
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"]   = to_email

    # 文本和 HTML 双格式（客户端若不支持 HTML 时也有纯文本）
    text = preamble + "\n\n" + df.to_string(index=False)
    html = f"""
    <html>
      <body>
        <p>{preamble}</p>
        {html_table}
      </body>
    </html>
    """

    msg.attach(MIMEText(text, "plain"))
    msg.attach(MIMEText(html, "html"))

    # 3) 登录 Gmail SMTP 并发送
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
        server.login(from_email, app_password)
        server.send_message(msg)

    print("邮件已发送。")
if __name__ == "__main__":
    update_local_csv()
    df_all = pd.read_csv("nkopmini_chain_with_dates.csv", parse_dates=['trade_date','expiry_date'])
    latest_date = df_all['trade_date'].max()
    df_today    = df_all[df_all['trade_date'] == latest_date].copy()
    # 3) 计算理论比值
    df_today = df_today[df_today['call_iv'].notna()]
    df_ratio = compute_theo_ratio(df_today, underlying_price=39340.0)
    # 4) 初步过滤
    df_filt = filter_candidates(df_ratio, theo_ratio_min=0.4, dte_limit=40)
    # 5) 打分并输出前 20
    df_top = score_candidates(df_filt, top_n=20)
    display(df_top[['trade_date','expiry_date','strike','call_settlement','theo_ratio',
                    'dte','call_oi','call_volume','call_delta','call_theta','call_gamma','call_vega','score']])
    df_send = df_top[['trade_date','expiry_date','strike','call_settlement','dte','call_volume','call_delta','call_theta','score']]
    
    #send_df_via_email_html(df_send,to_email="daiyutong970511@gmail.com",from_email="daiyutong970511@gmail.com",app_password="swhn btyt jnyy fhec",subject="今日期权候选列表",preamble="今日期权候选列表如下：")

更新完成，共有 4034 条记录，最新 trade_date：2025-07-14


,trade_date,expiry_date,strike,call_settlement,theo_ratio,dte,call_oi,call_volume,call_delta,call_theta,call_gamma,call_vega,score
0,2025-07-14,2025-08-07,39500.0,830.0,0.978721,24,229.0,106.0,0.4834,-14.6612,0.0002,43.4382,0.708341
1,2025-07-14,2025-08-07,41000.0,240.0,0.871840,24,266.0,302.0,0.2029,-9.5271,0.0001,30.7631,0.621159
2,2025-07-14,2025-08-07,40500.0,385.0,0.932082,24,373.0,106.0,0.2894,-11.7418,0.0002,37.2791,0.611836
3,2025-07-14,2025-08-07,40000.0,580.0,0.949009,24,180.0,54.0,0.3831,-13.6175,0.0002,41.5858,0.590869
4,2025-07-14,2025-08-07,39750.0,695.0,0.956080,24,73.0,5.0,0.4365,-14.3181,0.0002,42.9345,0.586763
5,2025-07-14,2025-08-07,39625.0,760.0,0.948150,24,17.0,5.0,0.4505,-14.7040,0.0002,43.0862,0.575027
6,2025-07-14,2025-08-07,39875.0,655.0,0.996668,24,18.0,2.0,0.4114,-13.8504,0.0002,42.4158,0.556724
7,2025-07-14,2025-08-07,40250.0,480.0,0.942925,24,70.0,69.0,0.3382,-12.8476,0.0002,39.8600,0.538330
8,2025-07-14,2025-08-07,40750.0,315.0,0.930812,24,45.0,80.0,0.2449,-10.6902,0.0002,34.2586,0.497714
9,2025-07-14,2025-08-07,40375.0,450.0,0.968471,24,35.0,2.0,0.3103,-12.3306,0.0002,38.4509,0.490728


In [2]:
#更新期权日级实际价格信息
MASTER_CSV  = "option_master.csv"
BASE_URL    = "https://www.jpx.co.jp"
INDEX_URL   = "https://www.jpx.co.jp/markets/derivatives/option-price/index.html"
# … 省略 fetch_latest_option_data 定义 …
# 如果 ZIP 解压后的 CSV 没有表头，就用下面的中文列名顺序
COLUMN_NAMES = [
    "商品代码", "商品类型", "限月", "行权价格（円/ポイント）", "预留",
    # 看跌期权
    "看跌_合约代码", "看跌_收盘价", "看跌_预留", "看跌_理论价格", "看跌_IV",
    # 看涨期权
    "看涨_合约代码", "看涨_收盘价", "看涨_预留", "看涨_理论价格", "看涨_IV",
    # 标的收盘价 & 基准波动率
    "标的收盘价", "基准波动率"
]

def fetch_latest_option_data(existing_max_date):
    """
    抓取 JPX 页面上所有 ZIP 链接，筛选取引日 > existing_max_date 的文件，
    下载并解压 CSV（无表头），并为其指定中文列名，返回 DataFrame（包含所有商品类型）。
    """
    resp = requests.get(INDEX_URL)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.lower().endswith(".zip"):
            parent_tr = a.find_parent("tr")
            if not parent_tr:
                continue
            tds = parent_tr.find_all("td")
            if not tds:
                continue
            date_text = tds[0].get_text(strip=True)
            try:
                date_obj = datetime.strptime(date_text, "%Y/%m/%d").date()
            except ValueError:
                continue
            full_url = href if href.startswith("http") else BASE_URL + href
            links.append((date_obj, full_url))

    new_links = [(d, url) for (d, url) in links if d > existing_max_date]
    if not new_links:
        return pd.DataFrame()

    data_frames = []
    for date_obj, zip_url in sorted(new_links):
        print(f"Downloading {zip_url} (交易日期: {date_obj}) ...")
        r = requests.get(zip_url)
        r.raise_for_status()
        with ZipFile(BytesIO(r.content)) as z:
            csv_names = [name for name in z.namelist() if name.lower().endswith(".csv")]
            if not csv_names:
                continue
            csv_name = csv_names[0]
            with z.open(csv_name) as csvfile:
                # 读取无表头 CSV，指定中文列名
                df = pd.read_csv(csvfile, encoding="shift_jis", header=None, names=COLUMN_NAMES, dtype=str)
                # 添加一列“交易日期”
                df["交易日期"] = date_obj
                data_frames.append(df)

    if not data_frames:
        return pd.DataFrame()

    return pd.concat(data_frames, ignore_index=True)
def update_master_csv():
    """
    更新本地的 option_master.csv：
    - 如果文件存在，读取最大“交易日期”，下载网页上大于该日期的 ZIP，解压并追加所有商品类型；
    - 如果文件不存在，则下载最新 ZIP 初始化并包含所有商品类型。
    """
    # 1) 安全读取：Shift_JIS 优先，失败后 utf-8-sig
    def safe_read(path):
        try:
            return pd.read_csv(path, parse_dates=["交易日期"], encoding="shift_jis")
        except UnicodeDecodeError:
            return pd.read_csv(path, parse_dates=["交易日期"], encoding="utf-8-sig")

    if os.path.exists(MASTER_CSV):
        # 2) 读已有文件
        master_df = safe_read(MASTER_CSV)
        # 保留为 Timestamp
        existing_max_ts = master_df["交易日期"].max()
        existing_max_date = existing_max_ts.date()  # 仅供 fetch 使用

        # 3) 获取新数据
        new_df = fetch_latest_option_data(existing_max_date)
        if new_df.empty:
            print("没有新数据。Master CSV 已是最新。")
            return

        # 4) 合并、去重
        combined = pd.concat([master_df, new_df], ignore_index=True)
        combined.drop_duplicates(
            subset=["交易日期", "限月", "行权价格（円/ポイント）", "商品类型"],
            keep="last",
            inplace=True
        )
        # 5) 强制时间列为 datetime，避免 date vs Timestamp 混用
        combined["交易日期"] = pd.to_datetime(combined["交易日期"])
        combined.sort_values(
            by=["交易日期", "限月", "行权价格（円/ポイント）"],
            inplace=True
        )

        # 6) 保存为 UTF-8 with BOM
        combined.to_csv(MASTER_CSV, index=False, encoding="utf-8-sig")
        print(f"Master CSV 已更新。新增 {len(new_df)} 行，总行数：{len(combined)}。")

    else:
        # 首次初始化
        new_df = fetch_latest_option_data(existing_max_date=datetime(1900,1,1).date())
        if new_df.empty:
            print("未从 JPX 页面获取到初始化数据。")
            return
        new_df["交易日期"] = pd.to_datetime(new_df["交易日期"])
        new_df.sort_values(
            by=["交易日期", "限月", "行权价格（円/ポイント）"],
            inplace=True
        )
        new_df.to_csv(MASTER_CSV, index=False, encoding="utf-8-sig")
        print(f"Master CSV 已初始化，共 {len(new_df)} 行。")

if __name__ == "__main__":
    update_master_csv()


Master CSV 已更新。新增 72234 行，总行数：431954。


In [4]:

# 请根据实际路径修改
MASTER_CSV_PATH = "option_master.csv"  # 或 "/mnt/data/option_master.csv"

# 1. 加载主文件
df_master = pd.read_csv(MASTER_CSV_PATH, encoding="utf-8-sig", parse_dates=["交易日期"])

# 2. 构建标准结构，包括 product_code 列
df_put = pd.DataFrame({
    "date": df_master["交易日期"],
    "contract_month": df_master["限月"],
    "strike": df_master["行权价格（円/ポイント）"],
    "underlying": df_master["标的收盘价"],
    "market_price": df_master["看跌_收盘价"],
    "theo_price": df_master["看跌_理论价格"],
    "iv_real": df_master["看跌_IV"],
    "option_type": "PUT",
    "product_code": df_master["商品代码"]
})

df_call = pd.DataFrame({
    "date": df_master["交易日期"],
    "contract_month": df_master["限月"],
    "strike": df_master["行权价格（円/ポイント）"],
    "underlying": df_master["标的收盘价"],
    "market_price": df_master["看涨_收盘价"],
    "theo_price": df_master["看涨_理论价格"],
    "iv_real": df_master["看涨_IV"],
    "option_type": "CALL",
    "product_code": df_master["商品代码"]
})

df_standard = pd.concat([df_put, df_call], ignore_index=True)

# 3. 225 主板 (NK225E) 与 225 mini (NK225MWE) 筛选
df_225 = df_standard[df_standard["product_code"].str.startswith("NK225E")]
df_225mini = df_standard[df_standard["product_code"].str.startswith("NK225MWE")]

# 4. 分别按类型拆分
df_225_call = df_225[df_225["option_type"] == "CALL"]
df_225_put  = df_225[df_225["option_type"] == "PUT"]
df_225mini_call = df_225mini[df_225mini["option_type"] == "CALL"]
df_225mini_put  = df_225mini[df_225mini["option_type"] == "PUT"]

# 5. 保存为 CSV
#df_225_call.to_csv("NK225_call.csv", index=False, encoding="utf-8-sig")
#df_225_put.to_csv("NK225_put.csv", index=False, encoding="utf-8-sig")
df_225mini_call.to_csv("NK225MWE_call.csv", index=False, encoding="utf-8-sig")
df_225mini_put.to_csv("NK225MWE_put.csv", index=False, encoding="utf-8-sig")


In [ ]:
#抓取历史数据（通过月数据）
# —— 配置 —— #
YEAR_CONFIG = {
    "2023": {
        "html": "https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/archives-2023.html",
        "json": None
    },
    "2024": {
        "html": "https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/archives-01.html",
        "json": "https://www.jpx.co.jp/automation/markets/statistics-derivatives/monthly_quotations/json/monthly_quotations_2024.json"
    },
    "2025": {
        "html": "https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/index.html",
        "json": "https://www.jpx.co.jp/automation/markets/statistics-derivatives/monthly_quotations/json/monthly_quotations_2025.json"
    },
}
MONTH_MAP = {
    "Jan":"01","Feb":"02","Mar":"03","Apr":"04",
    "May":"05","Jun":"06","Jul":"07","Aug":"08",
    "Sep":"09","Oct":"10","Nov":"11","Dec":"12"
}
# 25 列最终命名
FINAL_COLS = [
    "year_month","product_jp","option_type","flex_settlement","contract_month",
    "strike","open_date","open_price","open_jnet",
    "high_date","high_price","high_jnet",
    "low_date","low_price","low_jnet",
    "close_date","close_price","close_jnet",
    "avg_close_price","volume","trading_value",
    "exercised","open_interest","days_traded","last_trading_day"
]
OUTPUT = "hist_index_options_all.csv"
# —— 配置结束 —— #

def scrape_links():
    links = []
    base = "https://www.jpx.co.jp"
    pat_html = re.compile(r"SIOP_M_(\d{6})\.xlsx", re.IGNORECASE)
    pat_xlsx = re.compile(r"(\d{6})\.xlsx$", re.IGNORECASE)

    # 静态抓
    for year, cfg in YEAR_CONFIG.items():
        if cfg["html"]:
            print(f"[{year}] HTML 抓取")
            r = requests.get(cfg["html"]); r.raise_for_status()
            soup = BeautifulSoup(r.text, "html.parser")
            for a in soup.find_all("a", href=True):
                if pat_html.search(a["href"]):
                    ym = pat_html.search(a["href"]).group(1)
                    url = a["href"] if a["href"].startswith("http") else base + a["href"]
                    links.append((ym, url))

        # 静态无果，JSON 兜底
        if cfg["json"] and not any(l[0].startswith(year) for l in links):
            print(f"[{year}] JSON 兜底")
            r = requests.get(cfg["json"]); r.raise_for_status()
            data = r.json().get("TableDatas", [])
            for entry in data:
                for mth, mnum in MONTH_MAP.items():
                    link = entry.get(mth)
                    if link and link.endswith(".xlsx"):
                        ym = pat_xlsx.search(link).group(1)
                        full = link if link.startswith("http") else base + link
                        links.append((ym, full))

    # 去重按年月排序
    return sorted({(ym, url) for ym, url in links}, key=lambda x: x[0])

def download_all(links):
    raws = []
    for ym, url in links:
        print(f"下载 {ym} → {url}")
        r = requests.get(url); r.raise_for_status()
        # 不给 header，跳过前6行
        df0 = pd.read_excel(BytesIO(r.content), engine="openpyxl",
                            header=None, skiprows=6)
        # 在原表最后加年月列
        df0["year_month"] = ym
        raws.append(df0)
    return pd.concat(raws, ignore_index=True)


def main():
    # 1) 抓取所有候选链接（HTML + JSON 兜底）
    links = scrape_links()

    # 2) **只保留文件名里带 "SIOP" 的**，其余忽略
    links = [
        (ym, url) for ym, url in links
        if "SIOP" in os.path.basename(url).upper()
    ]
    print(f"过滤后，剩余 {len(links)} 个 SIOP 文件。")

    if not links:
        print("⚠️ 没有找到任何 SIOP 文件，程序退出。")
        return

    # 3) 下载（只会下载上面筛出来的 SIOP 文件）
    raw = download_all(links)

    # 4) 其余部分先不动，之后再给它随便插个列名就行
    #    比如直接：
    raw.to_csv("raw_siop_concat.csv", index=False, encoding="utf-8-sig")
    print("已将原始合并结果输出到 raw_siop_concat.csv")

if __name__ == "__main__":
    main()

[2023] HTML 抓取
[2024] HTML 抓取
[2024] JSON 兜底
[2025] HTML 抓取
[2025] JSON 兜底
过滤后，剩余 29 个 SIOP 文件。
下载 202301 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/fi1l5r0000001l53-att/SIOP_M_202301.xlsx
下载 202302 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/cg27su000000281w-att/SIOP_M_202302.xlsx
下载 202303 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/cg27su0000004hx5-att/SIOP_M_202303.xlsx
下载 202304 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/cg27su0000006gl7-att/SIOP_M_202304.xlsx
下载 202305 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/cg27su0000008ys8-att/SIOP_M_202305.xlsx
下载 202306 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/aocfb40000001xdx-att/SIOP_M_202306.xlsx
下载 202307 → https://www.jpx.co.jp/markets/statistics-derivatives/monthly-quotations/aocfb40000003v4e-att/SIOP_M_202307.xlsx
下载 202308 → https://www.jpx.co.jp/ma

In [5]:
# 1) 读取你刚才合并好的 CSV
df = pd.read_csv("raw_siop_concat.csv", dtype=str)
# 2) 删除列名为 '4' 和 '25' 的垃圾列
cols_to_drop = [c for c in ['4', '25'] if c in df.columns]
df = df.drop(columns=cols_to_drop)
# 3) 删除第一列
df.drop(columns=[df.columns[0]], inplace=True)
# 4) 先把所有单个字符 “－” 或 "-"（中英文短横）替换为 np.nan
df = df.replace({"－": np.nan, "-": np.nan})


In [2]:
# 1) 读取原始合并数据并清洗
raw = pd.read_csv("raw_siop_concat.csv", dtype=str)
# 删除无用列
for c in ['4', '25', raw.columns[0]]:
    if c in raw.columns:
        raw.drop(columns=c, inplace=True)
# 替换缺失符号
raw.replace({"－": np.nan, "-": np.nan}, inplace=True)

# 2) 过滤 Nikkei 225 Mini Options，从 2023-05 起
df = raw[(raw["product_id"] == "Nikkei 225 mini Options") &
         (raw["year_month.1"].astype(int) >= 202306)&
         (raw["option_type"] == "C")].copy()

# 3) 构建观测点表 obs_df
records = []
for _, row in df.iterrows():
    ym = str(int(row["year_month.1"]))
    year, month = int(ym[:4]), int(ym[4:6])
    for date_col, price_col in [
        ("open_date", "open_price"),
        ("high_date", "high_price"),
        ("low_date", "low_price"),
        ("close_date", "close_price")
    ]:
        day = row.get(date_col)
        price = row.get(price_col)
        if pd.notna(day) and pd.notna(price):
            try:
                dt = pd.Timestamp(year=year, month=month, day=int(day))
                price_val = float(price)
            except:
                continue
            records.append({
                "date": dt,
                "contract_month": pd.to_datetime(row["contract_month"]),
                "strike": float(row["strike"]),
                "option_type": row["option_type"],
                "market_price": price_val
            })
obs_df = pd.DataFrame(records)
# 1) 对 obs_df 去重并取平均值 防止日内波动引起重复
obs_df_dedup = obs_df.groupby(
    ['date', 'contract_month', 'strike', 'option_type'],
    as_index=False
).agg({
    'market_price': 'mean'
})


In [ ]:
# 2) 构建所有交易日期
# 读取期货交易日历，2023-05-01之后
fut = pd.read_excel("N225minif_2025.xlsx", sheet_name="日中日足", parse_dates=["日付"])
#fut = fut.rename(columns={"日付":"date"})
fut = fut.rename(columns={"日付":"date","終値":"underlying"})
trade_days = fut[fut["date"] >= "2023-06-01"]["date"].sort_values().unique()

# 3) 构建所有合约组合
#contracts = obs_df_dedup[['contract_month', 'strike', 'option_type']].drop_duplicates()
contracts = raw[
    (raw["product_id"]=="Nikkei 225 mini Options") &
    (raw["year_month.1"].astype(int)>=202306)&
         (raw["option_type"] == "C")
][["year_month.1","contract_month","strike","option_type"]].copy()
contracts["year_month.1"] = contracts["year_month.1"].astype(int)
contracts = contracts.drop_duplicates(subset=["year_month.1","contract_month","strike","option_type"])
# 4) 笛卡尔积日期 × 合约
all_dates = pd.DataFrame({'date': trade_days})
full = all_dates.merge(contracts, how='cross')
#除了“6 月数据出现 7 月合约” 的问题
full["ym"] = full["date"].dt.strftime("%Y%m").astype(int)
full = full[full["ym"] == full["year_month.1"]].drop(columns="ym")
# 5) 过滤已过期合约：只保留 date < contract_month
full = full[full['date'] < full['contract_month']]

# 6) 合并 market_price

# 先统一 full 里的 contract_month 为 datetime
# 1) 确保 full.contract_month 是 datetime
full["contract_month"] = pd.to_datetime(full["contract_month"])

# 2) 确保 full.strike 是数值型
full["strike"] = pd.to_numeric(full["strike"], errors="coerce")

# 3) 同步转换 obs_df_dedup  
obs_df_dedup["contract_month"] = pd.to_datetime(obs_df_dedup["contract_month"])
obs_df_dedup["strike"]         = pd.to_numeric(obs_df_dedup["strike"], errors="coerce")
full = full.merge(obs_df_dedup, on=['date','contract_month','strike','option_type'], how='left')

# 7) 排序并展示
#full = full.sort_values(['date','contract_month','strike','option_type']).reset_index(drop=True)
# 10) 合并 underlying
full = full.merge(fut[["date","underlying"]], on="date", how="left")
full = full.merge(fut[["date","underlying"]], on="date", how="left")
full["theo_price"] = pd.NA
full["iv_real"]    = pd.NA
#full.to_csv("NK225M_simulated_call_full_with_underlying_marketprice.csv", index=False)

In [27]:
#计算理论价格
from scipy.stats import linregress

# 1) 读取预处理后的全表 CSV
df = pd.read_csv("NK225M_simulated_call_full_with_underlying_marketprice.csv",
                 parse_dates=["date", "contract_month"])

# 2) 计算剩余到期时间 T（年）
df["dte"] = (df["contract_month"] - df["date"]).dt.days
df["T"] = df["dte"] / 252

# 3) 估算历史波动率：30日滚动年化
df = df.sort_values("date")
price_df = (
    df[['date','underlying']]
      .drop_duplicates(subset=['date'])
      .sort_values('date')
      .reset_index(drop=True)
)
price_df['log_ret'] = np.log(price_df['underlying'] / price_df['underlying'].shift(1))
price_df['hv30']    = price_df['log_ret'].rolling(30).std() * np.sqrt(252)
price_df['sigma']   = price_df['hv30'].bfill() 
df = df.merge(price_df[['date','sigma']], on='date', how='left')
# 4) 计算 Black-Scholes 理论价格
from scipy.stats import norm

def bs_call_price(S, K, T, sigma, r=0.005):
    """简化的无利率BS欧式看涨价格"""
    d1 = (np.log(S/K) + 0.5*sigma**2*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*norm.cdf(d2)

df["theo_price"] = bs_call_price(
    df["underlying"], df["strike"], df["T"], df["sigma"]
)

df = df.sort_values(["date","contract_month","strike","option_type"])




In [ ]:
#回归策略（有截距线性）
# 5) 提取 year_month 整数
df["year_month"] = df["date"].dt.strftime("%Y%m").astype(int)

# 6) 分层（月度）线性回归：market_price ~ theo_price
results = []
for ym, group in df.dropna(subset=["market_price"]).groupby("year_month"):
    x = group["theo_price"]
    y = group["market_price"]
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    results.append({
        "year_month": ym,
        "slope":       slope,
        "intercept":   intercept,
        "r_squared":   r_value**2,
        "n_points":    len(group)
    })
res_df = pd.DataFrame(results).sort_values("year_month")

# 7) 展示月度回归表现

# 8) 利用回归系数填充拟合 market_price
df = df.merge(res_df, on="year_month", how="left")
df["market_price_fitted"] = df["intercept"] + df["slope"] * df["theo_price"]

# 9) 边界截断：不超月内极值
# 先计算月内高低
extrema = df.groupby("year_month")["market_price"].agg(
    low_act=lambda s: s.min(),
    high_act=lambda s: s.max()
).reset_index()
df = df.merge(extrema, on="year_month", how="left")
df["market_price_fitted"] = df["market_price_fitted"].clip(
    lower=df["low_act"], upper=df["high_act"]
)

# 10) 输出带拟合价的结果前20行


In [7]:
#回归策略（无截距线性）
from sklearn.linear_model import LinearRegression
# 4) 分月无截距线性回归：market_price ~ theo_price, no intercept
# （假设 df 已经包含 date, theo_price, market_price 列 并已加载）
df["year_month"] = df["date"].dt.strftime("%Y%m").astype(int)

results_no_intercept = []
for ym, group in df.dropna(subset=["market_price","theo_price"]).groupby("year_month"):
    x = group["theo_price"].values.reshape(-1, 1)
    y = group["market_price"].values
    n = len(group)
    if n >= 10:
        lr = LinearRegression(fit_intercept=False)
        lr.fit(x, y)
        y_pred = lr.predict(x)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r2    = 1 - ss_res / ss_tot if ss_tot != 0 else np.nan
        slope = lr.coef_[0]
    else:
        slope, r2 = np.nan, np.nan
    results_no_intercept.append({
        "year_month":           ym,
        "slope_no_intercept":   slope,
        "r_squared_no_intercept": r2,
        "n_points":             n
    })

monthly_no_int = pd.DataFrame(results_no_intercept).sort_values("year_month")

In [ ]:
#回归策略（无截距线性+按月滚动）
# 2) 添加年月字段
df["year_month"] = df["date"].dt.to_period("M")

# 3) 保留有效散点
df_valid = df.dropna(subset=["theo_price", "market_price"])

# 4) 定义滚动窗口大小（月）
WINDOW = 1

# 5) 获取所有月份
months = sorted(df_valid["year_month"].unique())

# 6) 滚动窗口回归
results = []
for i, m in enumerate(months):
    # 窗口起始索引
    start_idx = max(0, i - WINDOW + 1)
    window_months = months[start_idx:i+1]
    sub = df_valid[df_valid["year_month"].isin(window_months)]
    x = sub["theo_price"].values.reshape(-1, 1)
    y = sub["market_price"].values
    n = len(sub)
    if n >= 10:
        lr = LinearRegression(fit_intercept=True)
        lr.fit(x, y)
        y_pred = lr.predict(x)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r2 = 1 - ss_res/ss_tot if ss_tot != 0 else np.nan
        slope = lr.coef_[0]
        intercept = lr.intercept_
    else:
        slope, intercept, r2, n = np.nan, np.nan, np.nan, n
    results.append({
        "year_month": m.to_timestamp(),
        "slope_rw": slope,
        "intercept_rw": intercept,
        "r_squared_rw": r2,
        "n_points": n
    })

monthly_rw = pd.DataFrame(results)

In [28]:
#回归策略（无截距线性+低拟合度按中位数填充）
# 2) 添加年月字段
df["year_month"] = df["date"].dt.strftime("%Y%m").astype(int)

# 3) 计算月度中位数比例
#    ratio = market_price / theo_price（仅在两者均非空时计算）
df["ratio"] = df["market_price"] / df["theo_price"]
median_ratio = df.dropna(subset=["ratio"]).groupby("year_month")["ratio"]\
                 .median().rename("median_ratio").reset_index()

# 4) 分月无截距线性回归：market_price ~ theo_price, no intercept
# （假设 df 已经包含 date, theo_price, market_price 列 并已加载）

results_no_intercept = []
for ym, group in df.dropna(subset=["market_price","theo_price"]).groupby("year_month"):
    x = group["theo_price"].values.reshape(-1, 1)
    y = group["market_price"].values
    n = len(group)
    if n >= 10:
        lr = LinearRegression(fit_intercept=False)
        lr.fit(x, y)
        y_pred = lr.predict(x)
        ss_res = np.sum((y - y_pred) ** 2)
        ss_tot = np.sum((y - np.mean(y)) ** 2)
        r2    = 1 - ss_res / ss_tot if ss_tot != 0 else np.nan
        slope = lr.coef_[0]
    else:
        slope, r2 = np.nan, np.nan
    results_no_intercept.append({
        "year_month":           ym,
        "slope":   slope,
        "r_squared": r2,
        "n_points":             n
    })

monthly_no_int = pd.DataFrame(results_no_intercept)

# 5) 合并回主表
df = df.merge(monthly_no_int, on="year_month", how="left")
df = df.merge(median_ratio, on="year_month", how="left")

# 6) 根据阈值选择拟合方式
R2_THRESHOLD = 0.5
df["market_price_fitted"] = np.where(
    df["r_squared"] >= R2_THRESHOLD,
    df["theo_price"] * df["slope"],          # 无截距线性拟合
    df["theo_price"] * df["median_ratio"]    # 中位数比例回退
)

# 9) 边界截断：不超月内极值
# 先计算月内高低
extrema = df.groupby("year_month")["market_price"].agg(
    low_act=lambda s: s.min(),
    high_act=lambda s: s.max()
).reset_index()
df = df.merge(extrema, on="year_month", how="left")
df["market_price_fitted"] = df["market_price_fitted"].clip(
    lower=df["low_act"], upper=df["high_act"]
)

df["market_price"] = df["market_price"].fillna(df["market_price_fitted"])
# 3) 选取回测需要的字段
backtest_df = df[[
    "date",
    "contract_month",
    "strike",
    "underlying",
    "market_price",
    "theo_price",
    "iv_real"
]].copy()
# 4) 导出为 CSV
backtest_df.to_csv("NK225MWE_call_backtest_2023_fitted.csv", index=False)
print(backtest_df.head(20))

         date contract_month  strike  underlying  market_price  theo_price  \
0  2023-06-01     2023-06-02   30625       31150        285.00  536.142416   
1  2023-06-01     2023-06-02   30750       31150        291.25  423.611504   
2  2023-06-01     2023-06-02   30875       31150        187.00  320.543942   
3  2023-06-01     2023-06-02   31000       31150        141.25  230.493254   
4  2023-06-01     2023-06-02   31125       31150         74.00  156.300553   
5  2023-06-01     2023-06-02   31250       31150         43.00   99.242693   
6  2023-06-01     2023-06-02   31375       31150         22.75   58.628685   
7  2023-06-01     2023-06-02   31500       31150         10.50   32.049594   
8  2023-06-01     2023-06-02   31625       31150          5.25   16.138447   
9  2023-06-01     2023-06-02   31750       31150          2.50    7.458149   
10 2023-06-01     2023-06-02   31875       31150          1.25    3.154074   
11 2023-06-01     2023-06-02   32000       31150          1.00  

In [29]:
df["r_squared"].drop_duplicates()

0         0.806476
4990      0.876008
9716      0.894446
15533     0.911272
20609     0.841099
26168     0.832521
31040     0.897311
35734     0.826806
40900     0.800731
46566     0.769038
51999     0.836859
58423     0.857947
64476     0.878986
68904     0.824420
75996     0.165886
83306     0.373682
89259     0.810347
95388     0.790807
100291    0.869492
105686    0.685445
110727    0.869588
115898    0.827075
121882    0.230353
129331    0.379521
Name: r_squared, dtype: float64

In [30]:
#添加真实数据
# 1. 读取拟合数据（用于回测）
df_fit = pd.read_csv("NK225MWE_call_backtest_2023_fitted.csv", parse_dates=["date"])
df_fit['contract_month'] = pd.to_datetime(df_fit['contract_month'])
df_fit = df_fit.set_index("date")

# 2. 读取真实数据
df_real = pd.read_csv("NK225MWE_call.csv", parse_dates=["date"])
df_real['contract_month'] = pd.to_datetime(
    df_real['contract_month'].astype(str),
    format="%Y%m%d"
)
df_real = df_real.set_index("date")

# 3. 只保留 2025-03-26 以后的真实数据，并按拟合数据的列顺序重排
start_replace = pd.to_datetime("2025-03-26")
df_sub = df_real.loc[start_replace:]
# 确保列名一致、顺序与 df_fit 相同
df_sub = df_sub.reindex(columns=df_fit.columns)


# 4. 分割：保留拟合数据中 2025-03-26 之前的部分
df_pre = df_fit.loc[df_fit.index < start_replace]


# 6. 直接拼接
df_final = pd.concat([df_pre, df_sub])

# 7. 恢复索引为列（可选）
df_final = df_final.reset_index()

# 6. 保存或后续使用
df_final.to_csv("NK225MWE_call_backtest_2023_fitted_2503updated.csv", index=False)

# 日经225迷你期权溢价covered call卖出策略(无期货信号入场测试版)

## 参数
- 溢价阈值：`市价/理论价 ≥ 1.0` 卖出
- 止损阈值：`浮亏 ≥ 权利金×40%` 双边止损
- 合约乘数：`100` mini

## 策略逻辑
- **持仓时**  
  1. 计算期货浮动盈亏  
  2. 若浮亏触及止损，按当日市价平仓并记录  
  3. 到期放弃期权，收回全部权利金并记录  
- **空仓时**  
  1. 计算每笔合约的溢价比和剩余天数  
  2. 筛选：溢价高、剩余≤40天、行权价高于标的  
  3. 从中选市价最高合约建仓  

## 关键点
- **止损保护**：及时平仓，防止大亏  
- **到期收益**：放弃期权即获全部权利金  
- **溢价优选**：用价差和剩余天数控制时机，优选高溢价合约  

In [ ]:
#无期货信号版（基准测试）
# 1) 读取期权数据并筛选测试区间
#opt_df = pd.read_csv("NK225M_call_noniv_withcountprice.csv", parse_dates=["date", "contract_month"])
#opt_df = pd.read_csv("NK225MWE_call.csv", parse_dates=["date", "contract_month"])
#opt_df = pd.read_csv("NK225MWE_call_backtest_2023_fitted.csv", parse_dates=["date", "contract_month"])
opt_df = pd.read_csv("NK225MWE_call_backtest_2023_fitted_2503updated.csv", parse_dates=["date", "contract_month"])

#start_date = pd.to_datetime("2023-03-01")
#end_date   = pd.to_datetime("2024-07-31")
#start_date = pd.to_datetime("2024-10-01")
#end_date   = pd.to_datetime("2025-06-16")
#opt_df = opt_df[(opt_df['date'] >= start_date) & (opt_df['date'] <= end_date)].copy()
# 2. 排除 2024-08 和 2024-09
mask = opt_df['date'].dt.to_period("M").isin(["2024-08", "2024-09"])
opt_df = opt_df[~mask]
# 2) 策略参数（可调整）
THEO_RATIO      = 0.8      # 市价/理论价 >= 1.2 时卖出
STOP_LOSS_RATIO = 0.4      # 如果期权价格下跌至原价的 (1 - STOP_LOSS_RATIO) 时止损
VALUE_PER_POINT = 100      # 合约乘数

# 3) 初始化日志列表与持仓状态
records = []
position_active = False
entry = {}

# 4) 逐日模拟策略
for today in sorted(opt_df['date'].unique()):
    today_data = opt_df[opt_df['date'] == today]
    
    # 4.1 持仓中：先判止损，再判到期
    if position_active:
        # 计算期货浮动 PnL（单位是合约乘数前的点值）
        current_fut = today_data['underlying'].iloc[0]
        fut_pnl = (current_fut - entry['price']) * VALUE_PER_POINT
        
        # 止损触发：当期货浮亏 ≥ 权利金 * STOP_LOSS_RATIO
        if -fut_pnl >= entry['opt_in'] * STOP_LOSS_RATIO:
            exit_opt_price = (
                today_data.loc[
                    today_data['strike'] == entry['strike'],
                    'market_price'
                ].iloc[0]
                * VALUE_PER_POINT
            )
            stop_loss_amt = -fut_pnl
            net_profit    = entry['opt_in'] - exit_opt_price - stop_loss_amt
            records.append({
                'entry_date': entry['date'],
                'exit_date': today,
                'contract_month':   entry['end_date'],
                'entry_opt_price': entry['opt_in'],
                'exit_opt_price': exit_opt_price,
                'stop_loss_amt': stop_loss_amt,
                'net_profit': net_profit,
                'return_pct': net_profit / entry['opt_in'],
                'reason': 'Stop Loss'
            })
            position_active = False
            entry = {}
            continue
        
        # 到期处理：期权作废，获得全部权利金
        if today >= entry['end_date']:
            net_profit = entry['opt_in']
            records.append({
                'entry_date': entry['date'],
                'exit_date': today,
                'contract_month':   entry['end_date'],
                'entry_opt_price': entry['opt_in'],
                'exit_opt_price': 0.0,
                'stop_loss_amt': 0.0,
                'net_profit': net_profit,
                'return_pct': 1.0,
                'reason': 'Expired'
            })
            position_active = False
            entry = {}
            continue

    # 4.2 无持仓：判入场信号
    if not position_active:
        today_data['theo_ratio'] = today_data['market_price'] / today_data['theo_price']
        today_data['dte'] = (today_data['contract_month'] - today_data['date']).dt.days
        cands = today_data[
            (today_data['theo_ratio'] >= THEO_RATIO) &
            (today_data['strike']    > today_data['underlying'])&
            (today_data['dte']       <= 40) 
        ]
        if not cands.empty:
            # 取权利金（market_price）最大的合约
            sel = cands.sort_values('market_price', ascending=False).iloc[0]
            entry = {
                'date':      today,
                'price':     today_data['underlying'].iloc[0],
                'opt_in':    sel['market_price'] * VALUE_PER_POINT,
                'strike':    sel['strike'],
                'end_date':  sel['contract_month']
            }
            position_active = True

# 5) 构建结果 DataFrame
result_df = pd.DataFrame(records)
result_df['exit_date'] = pd.to_datetime(result_df['exit_date'])

# 构造要剔除的年份和月份
years = [2024]
months = [8, 9]

# 过滤掉 exit_date 在 2024 年 8、9 月的记录
mask = ~(
    (result_df['exit_date'].dt.year.isin(years)) &
    (result_df['exit_date'].dt.month.isin(months))
)
result_df_filtered = result_df.loc[mask].reset_index(drop=True)
print(result_df_filtered)
print(result_df_filtered["net_profit"].sum())

   entry_date  exit_date contract_month  entry_opt_price  exit_opt_price  \
0  2023-06-01 2023-06-23     2023-06-23     48364.056818        0.000000   
1  2023-06-26 2023-07-07     2023-07-14     45619.558496    17528.399938   
2  2023-07-10 2023-08-11     2023-08-11     73500.000000        0.000000   
3  2023-08-14 2023-08-16     2023-08-18     61500.000000     1842.971152   
4  2023-08-17 2023-09-15     2023-09-15     50085.161391        0.000000   
5  2023-09-18 2023-09-21     2023-10-20     48987.990667      100.000000   
6  2023-09-22 2023-09-26     2023-10-13     41836.384985     6783.378577   
7  2023-09-27 2023-09-28     2023-10-13     33631.923402      400.000000   
8  2023-09-29 2023-10-02     2023-10-20     40000.000000    23000.000000   
9  2023-10-03 2023-10-04     2023-11-10     80500.000000      100.000000   
10 2023-10-05 2023-10-26     2023-11-10     63018.249564      300.000000   
11 2023-10-27 2023-10-30     2023-11-10     43071.640811     9855.094667   
12 2023-10-3

In [42]:
result_df_filtered

,entry_date,exit_date,contract_month,entry_opt_price,exit_opt_price,stop_loss_amt,net_profit,return_pct,reason
0,2023-06-01,2023-06-23,2023-06-23,48364.056818,0.000000,0.0,48364.056818,1.000000,Expired
1,2023-06-26,2023-07-07,2023-07-14,45619.558496,17528.399938,22000.0,6091.158557,0.133521,Stop Loss
2,2023-07-10,2023-08-11,2023-08-11,73500.000000,0.000000,0.0,73500.000000,1.000000,Expired
3,2023-08-14,2023-08-16,2023-08-18,61500.000000,1842.971152,36000.0,23657.028848,0.384667,Stop Loss
4,2023-08-17,2023-09-15,2023-09-15,50085.161391,0.000000,0.0,50085.161391,1.000000,Expired
5,2023-09-18,2023-09-21,2023-10-20,48987.990667,100.000000,64500.0,-15612.009333,-0.318691,Stop Loss
6,2023-09-22,2023-09-26,2023-10-13,41836.384985,6783.378577,18500.0,16553.006408,0.395661,Stop Loss
7,2023-09-27,2023-09-28,2023-10-13,33631.923402,400.000000,30000.0,3231.923402,0.096097,Stop Loss
8,2023-09-29,2023-10-02,2023-10-20,40000.000000,23000.000000,19500.0,-2500.000000,-0.062500,Stop Loss
9,2023-10-03,2023-10-04,2023-11-10,80500.000000,100.000000,80000.0,400.000000,0.004969,Stop Loss


In [23]:
#期货信号开仓版
import pandas as pd
from datetime import datetime

# 1) 读取期权数据
opt_df = pd.read_csv("NK225MWE_call_backtest_2023_fitted_2503updated.csv", parse_dates=["date", "contract_month"])

mask = opt_df['date'].dt.to_period("M").isin(["2024-08", "2024-09"])
opt_df = opt_df[~mask]

# 2) 读取并生成期货信号（假设已保存或实时计算好）
#    df_sig.index 为 datetime，含列 ['front_settle','can_enter']
#    你可以直接从上一段代码得到的 df_sig 导入，或重复计算
# df_sig = generate_signals(compute_indicators(load_futures_excel(...)))

# 3) 回测参数
THEO_RATIO      = 0.4
STOP_LOSS_RATIO = 0.4
VALUE_PER_POINT = 100

records = []
position_active = False
entry = {}

for today in sorted(opt_df['date'].unique()):
    today_opt = opt_df[opt_df['date'] == today]
    
    # 持仓时：止损 & 到期
    if position_active:
        current_fut = today_opt['underlying'].iloc[0]
        fut_pnl = (current_fut - entry['price']) * VALUE_PER_POINT
        
        # 止损逻辑（浮亏 ≥ 权利金×STOP_LOSS_RATIO）
        if -fut_pnl >= entry['opt_in'] * STOP_LOSS_RATIO:
            # 获取当日退出期权市价
            exit_opt_price = today_opt.loc[
                today_opt['strike'] == entry['strike'],
                'market_price'
            ].iloc[0] * VALUE_PER_POINT
            stop_loss_amt = -fut_pnl
            net_profit    = entry['opt_in'] - exit_opt_price - stop_loss_amt
            records.append({
                'entry_date': entry['date'],
                'exit_date': today,
                'contract_month': entry['end_date'],
                'entry_opt_price': entry['opt_in'],
                'exit_opt_price': exit_opt_price,
                'stop_loss_amt': stop_loss_amt,
                'net_profit': net_profit,
                'return_pct': net_profit / entry['opt_in'],
                'reason': 'Stop Loss'
            })
            position_active = False
            entry = {}
            continue
        
        # 到期逻辑
        if today >= entry['end_date']:
            net_profit = entry['opt_in']
            records.append({
                'entry_date': entry['date'],
                'exit_date': today,
                'contract_month': entry['end_date'],
                'entry_opt_price': entry['opt_in'],
                'exit_opt_price': 0.0,
                'stop_loss_amt': 0.0,
                'net_profit': net_profit,
                'return_pct': 1.0,
                'reason': 'Expired'
            })
            position_active = False
            entry = {}
            continue

    # 空仓时：使用期货信号 + 溢价筛选入场
    if not position_active and today in df_sig.index and df_sig.at[today, 'can_enter']:
        # 计算理论溢价
        today_opt['theo_ratio'] = today_opt['market_price'] / today_opt['theo_price']
        # 筛选合约
        cands = today_opt[
            (today_opt['theo_ratio'] >= THEO_RATIO) &
            (today_opt['strike'] >= today_opt['underlying']) &
            ((today_opt['contract_month'] - today_opt['date']).dt.days <= 40)
        ]
        if not cands.empty:
            sel = cands.sort_values('market_price', ascending=False).iloc[0]
            entry = {
                'date': today,
                'price': df_sig.at[today, 'front_settle'],
                'opt_in': sel['market_price'] * VALUE_PER_POINT,
                'strike': sel['strike'],
                'end_date': sel['contract_month']
            }
            position_active = True

# 输出结果
result_df = pd.DataFrame(records)
result_df['exit_date'] = pd.to_datetime(result_df['exit_date'])

# 构造要剔除的年份和月份
years = [2024]
months = [8, 9]

# 过滤掉 exit_date 在 2024 年 8、9 月的记录
mask = ~(
    (result_df['exit_date'].dt.year.isin(years)) &
    (result_df['exit_date'].dt.month.isin(months))
)
result_df_filtered_2 = result_df.loc[mask].reset_index(drop=True)
print(result_df_filtered_2)
print(result_df_filtered_2["net_profit"].sum())

   entry_date  exit_date contract_month  entry_opt_price  exit_opt_price  \
0  2023-06-01 2023-06-23     2023-06-23     49615.385958        0.000000   
1  2023-07-03 2023-07-04     2023-08-11     72000.000000     6033.485193   
2  2023-07-05 2023-07-06     2023-08-11     69986.291755      100.000000   
3  2023-07-19 2023-07-20     2023-08-11     51638.420959      100.000000   
4  2023-07-21 2023-08-11     2023-08-11     46036.318367        0.000000   
5  2023-08-15 2023-08-16     2023-09-08     50629.790589     2875.288911   
6  2023-08-24 2023-08-25     2023-09-29     37915.461623     2614.088427   
7  2023-08-28 2023-09-15     2023-09-15     34261.036630        0.000000   
8  2023-09-19 2023-09-21     2023-10-20     49251.809896      100.000000   
9  2023-09-25 2023-09-26     2023-10-13     41397.143516     5602.467849   
10 2023-10-12 2023-10-13     2023-11-17     71396.438786    17373.745998   
11 2023-10-17 2023-10-19     2023-11-17     69270.311049      100.000000   
12 2023-11-0

In [24]:
result_df_filtered_2

,entry_date,exit_date,contract_month,entry_opt_price,exit_opt_price,stop_loss_amt,net_profit,return_pct,reason
0,2023-06-01,2023-06-23,2023-06-23,49615.385958,0.000000,0.0,49615.385958,1.000000,Expired
1,2023-07-03,2023-07-04,2023-08-11,72000.000000,6033.485193,32500.0,33466.514807,0.464813,Stop Loss
2,2023-07-05,2023-07-06,2023-08-11,69986.291755,100.000000,59000.0,10886.291755,0.155549,Stop Loss
3,2023-07-19,2023-07-20,2023-08-11,51638.420959,100.000000,56500.0,-4961.579041,-0.096083,Stop Loss
4,2023-07-21,2023-08-11,2023-08-11,46036.318367,0.000000,0.0,46036.318367,1.000000,Expired
5,2023-08-15,2023-08-16,2023-09-08,50629.790589,2875.288911,51500.0,-3745.498322,-0.073978,Stop Loss
6,2023-08-24,2023-08-25,2023-09-29,37915.461623,2614.088427,59000.0,-23698.626804,-0.625039,Stop Loss
7,2023-08-28,2023-09-15,2023-09-15,34261.036630,0.000000,0.0,34261.036630,1.000000,Expired
8,2023-09-19,2023-09-21,2023-10-20,49251.809896,100.000000,63000.0,-13848.190104,-0.281171,Stop Loss
9,2023-09-25,2023-09-26,2023-10-13,41397.143516,5602.467849,40000.0,-4205.324333,-0.101585,Stop Loss


In [25]:
#查看胜率盈亏比
result_df = result_df_filtered_2
total_trades = len(result_df)
wins = result_df[result_df['net_profit'] > 0]
losses = result_df[result_df['net_profit'] < 0]

win_rate = len(wins) / total_trades
avg_win = wins['net_profit'].mean() if not wins.empty else 0
avg_loss = losses['net_profit'].abs().mean() if not losses.empty else 0

profit_factor = wins['net_profit'].sum() / losses['net_profit'].abs().sum() if not losses.empty else float('inf')
win_loss_ratio = avg_win / avg_loss if avg_loss > 0 else float('inf')

# 汇总指标
metrics = pd.DataFrame({
    '总交易次数': [total_trades],
    '胜利次数': [len(wins)],
    '失败次数': [len(losses)],
    '胜率': [win_rate],
    '平均盈利': [avg_win],
    '平均亏损': [avg_loss],
    '盈亏比(平均盈利/平均亏损)': [win_loss_ratio],
    'Profit Factor(盈利总和/亏损总和)': [profit_factor]
})
metrics

,总交易次数,胜利次数,失败次数,胜率,平均盈利,平均亏损,盈亏比(平均盈利/平均亏损),Profit Factor(盈利总和/亏损总和)
0,42,32,10,0.761905,34725.207729,27869.821097,1.245979,3.987132


In [12]:
#可视化部分

import plotly.graph_objects as go


# 假设 opt_df 和 result_df 已经存在
# opt_df 包含 ['date','underlying']
# result_df 包含 ['entry_date','exit_date','reason']

# 1. 构造期货价格序列
fut_series = opt_df.drop_duplicates('date').set_index('date')['underlying']
result_df = result_df_filtered_2
# 2. 创建交互式图表
fig = go.Figure()

# 画期货价格折线
fig.add_trace(go.Scatter(
    x=fut_series.index,
    y=fut_series.values,
    mode='lines',
    name='期货价格'
))

# 画进场点
fig.add_trace(go.Scatter(
    x=result_df['entry_date'],
    y=result_df['entry_date'].map(fut_series),
    mode='markers',
    marker=dict(symbol='circle', size=8),
    name='进场'
))

# 画止损退出点
stop = result_df[result_df['reason']=='Stop Loss']
fig.add_trace(go.Scatter(
    x=stop['exit_date'],
    y=stop['exit_date'].map(fut_series),
    mode='markers',
    marker=dict(symbol='triangle-up', size=10),
    name='止损退出'
))

# 画到期退出点
exp = result_df[result_df['reason']!='Stop Loss']
fig.add_trace(go.Scatter(
    x=exp['exit_date'],
    y=exp['exit_date'].map(fut_series),
    mode='markers',
    marker=dict(symbol='square', size=10),
    name='到期退出'
))

# 更新布局
fig.update_layout(
    title="日经225迷你期货价格与交易标记（交互式）",
    xaxis_title="日期",
    yaxis_title="价格",
    hovermode='x unified'
)

fig.show()

In [11]:
#策略可视化对比
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 假设已准备好：
# fut_series: pd.Series，索引为 date，值为 underlying
# result_df1, result_df2: 两个 pd.DataFrame，各含 ['entry_date','exit_date','reason']
# 1. 构造期货价格序列
fut_series = opt_df.drop_duplicates('date').set_index('date')['underlying']
# 1. 创建 2 行 1 列的子图
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=("策略 A", "策略 B"),
    shared_xaxes=True
)

def add_strategy_traces(fig, result_df, row, name_suffix):
    # 期货价格
    fig.add_trace(
        go.Scatter(
            x=fut_series.index,
            y=fut_series.values,
            mode='lines',
            name=f'期货价格{name_suffix}',
            legendgroup=name_suffix
        ),
        row=row, col=1
    )
    # 进场
    fig.add_trace(
        go.Scatter(
            x=result_df['entry_date'],
            y=result_df['entry_date'].map(fut_series),
            mode='markers',
            marker_symbol='circle',
            marker_size=8,
            name=f'进场{name_suffix}',
            legendgroup=name_suffix
        ),
        row=row, col=1
    )
    # 止损退出
    stop = result_df[result_df['reason']=="Stop Loss"]
    fig.add_trace(
        go.Scatter(
            x=stop['exit_date'],
            y=stop['exit_date'].map(fut_series),
            mode='markers',
            marker_symbol='triangle-up',
            marker_size=10,
            name=f'止损退出{name_suffix}',
            legendgroup=name_suffix
        ),
        row=row, col=1
    )
    # 到期退出
    exp = result_df[result_df['reason']!="Stop Loss"]
    fig.add_trace(
        go.Scatter(
            x=exp['exit_date'],
            y=exp['exit_date'].map(fut_series),
            mode='markers',
            marker_symbol='square',
            marker_size=10,
            name=f'到期退出{name_suffix}',
            legendgroup=name_suffix
        ),
        row=row, col=1
    )

# 2. 分别添加两组策略
add_strategy_traces(fig, result_df_filtered_1, row=1, name_suffix="(A)")
add_strategy_traces(fig, result_df_filtered_2, row=2, name_suffix="(B)")

# 3. 更新布局
fig.update_layout(
    title="策略对比：日经225迷你期货交易标记",
    xaxis_title="日期",
    yaxis_title="价格",
    hovermode='x unified',
    height=800,
    legend=dict(tracegroupgap=50)
)

fig.update_xaxes(matches='x')  # 同步两行的 x 轴缩放
fig.show()


NameError: name 'result_df_filtered_1' is not defined

In [16]:
#网格筛选
# ---------- 1. 回测函数（沿用你之前的逻辑） ----------
def backtest(opt_df, df_sig,
             theo_ratio=0.4, stop_loss=0.2, point=100):
    records = []
    position_active = False
    entry = {}

    for today in sorted(opt_df['date'].unique()):
        today_opt = opt_df[opt_df['date'] == today]

        # 持仓：止损 & 到期
        if position_active:
            fut = today_opt['underlying'].iloc[0]
            fut_pnl = (fut - entry['price']) * point

            # 止损
            if -fut_pnl >= entry['opt_in'] * stop_loss:
                exit_opt = today_opt.loc[
                    today_opt['strike'] == entry['strike'],
                    'market_price'
                ].iloc[0] * point
                sl_amt = -fut_pnl
                net = entry['opt_in'] - exit_opt - sl_amt
                records.append({'net': net, 'win': net > 0})
                position_active = False
                entry = {}
                continue

            # 到期
            if today >= entry['end_date']:
                records.append({'net': entry['opt_in'], 'win': True})
                position_active = False
                entry = {}
                continue

        # 空仓：纯动量+波动率信号
        if (not position_active
            and today in df_sig.index
            and df_sig.at[today, 'can_enter']):
            today_opt = today_opt.copy()
            today_opt['theo_ratio'] = (
                today_opt['market_price'] / today_opt['theo_price']
            )
            cands = today_opt[
                (today_opt['theo_ratio'] >= theo_ratio) &
                (today_opt['strike'] > today_opt['underlying']) &
                ((today_opt['contract_month'] - today_opt['date']).dt.days <= 40)
            ]
            if not cands.empty:
                sel = cands.sort_values('market_price', ascending=False).iloc[0]
                entry = {
                    'date':      today,
                    'price':     df_sig.at[today, 'front_settle'],
                    'opt_in':    sel['market_price'] * point,
                    'strike':    sel['strike'],
                    'end_date':  sel['contract_month']
                }
                position_active = True

    df = pd.DataFrame(records)
    return {
        'net_profit': df['net'].sum(),
        'win_rate':   df['win'].mean() if len(df) else np.nan,
        'trades':     len(df)
    }

# ---------- 2. 指标与信号 ----------
def compute_indicators(df, mom_period, atr_short, atr_long):
    df = df.copy()
    df['Mom'] = df['front_settle'].pct_change(mom_period)
    high, low, prev = df['front_high'], df['front_low'], df['front_settle'].shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev).abs(),
        (low - prev).abs()
    ], axis=1).max(axis=1)
    df['ATR_short'] = tr.rolling(atr_short).mean()
    df['ATR_long']  = tr.rolling(atr_long).mean()
    return df.dropna(subset=['Mom','ATR_short','ATR_long'])

def generate_signals(df_ind, mom_thresh, atr_ratio):
    df = df_ind.copy()
    df['fail_reason'] = np.where(
        df['Mom'] <= mom_thresh,
        '强下跌',
        np.where(
            df['ATR_short'] >= df['ATR_long'] * atr_ratio,
            '超高波动',
            ''
        )
    )
    df['can_enter'] = df['fail_reason'] == ''
    return df


# ---------- 3. 加载数据 ----------
opt_df = pd.read_csv(
    "NK225MWE_call_backtest_2023_fitted_2503updated.csv",
    parse_dates=["date","contract_month"]
)
# 排除2024-08、09
opt_df = opt_df[~opt_df['date'].dt.to_period("M")
                .isin([pd.Period("2024-08"), pd.Period("2024-09")])]

df_raw = load_futures_excel(
    excel_path="N225minif_2025.xlsx",
    sheet_name="日中日足",
    date_col="日付",
    start_date="2023-06-01",
    end_date="2025-06-26"
)

# ---------- 4. 批量网格测试（加 theo_ratio、stop_loss） ----------
mom_periods   = [8,10,12]
atr_ratios    = [2.0,2.5]
#theo_ratios   = [0.4,0.5]
stop_losses   = [0.3, 0.2]
atr_shorts  = [14, 20]     # 要测试的 ATR 短期窗口
atr_longs   = [50, 60]    # 要测试的 ATR 长期窗口

results = []
for mom_p, atr_r, atr_s, atr_l in product(mom_periods, atr_ratios, atr_shorts, atr_longs):
    # 计算指标与信号
    df_ind = compute_indicators(df_raw, mom_period=mom_p, atr_short=atr_s, atr_long=atr_l)
    df_sig = generate_signals(df_ind, mom_thresh=0.0, atr_ratio=atr_r)
    # 回测
    perf   = backtest(opt_df, df_sig)
    perf.update({
        'mom_period': mom_p,
        'atr_ratio':  atr_r,
        'atr_short':  atr_s,
        'atr_long':   atr_l
    })
    results.append(perf)

# 输出结果
df_res = pd.DataFrame(results)
df_res = df_res.sort_values('net_profit', ascending=False).reset_index(drop=True)
print(df_res)

      net_profit  win_rate  trades  mom_period  atr_ratio  atr_short  atr_long
0   1.092221e+06  0.853659      41          10        2.5         14        50
1   1.092221e+06  0.853659      41          10        2.5         20        60
2   1.092221e+06  0.853659      41          10        2.5         20        50
3   1.092221e+06  0.853659      41          10        2.5         14        60
4   1.092221e+06  0.853659      41          10        2.0         14        50
5   1.092221e+06  0.853659      41          10        2.0         14        60
6   1.092221e+06  0.853659      41          10        2.0         20        50
7   1.092221e+06  0.853659      41          10        2.0         20        60
8   9.122085e+05  0.804878      41           8        2.0         14        60
9   9.122085e+05  0.804878      41           8        2.0         20        60
10  9.122085e+05  0.804878      41           8        2.5         14        60
11  9.122085e+05  0.804878      41           8      

In [ ]:
import numpy as np
import pandas as pd
from itertools import product

# ———— 1. 回测函数（保持不变） ————
def backtest(opt_df, df_sig,
             theo_ratio=0.4, stop_loss=0.2, point=100):
    records = []
    position_active = False
    entry = {}

    for today in sorted(opt_df['date'].unique()):
        today_opt = opt_df[opt_df['date'] == today]

        # 持仓：止损 & 到期
        if position_active:
            fut = today_opt['underlying'].iloc[0]
            fut_pnl = (fut - entry['price']) * point

            # 止损
            if -fut_pnl >= entry['opt_in'] * stop_loss:
                exit_opt = today_opt.loc[
                    today_opt['strike'] == entry['strike'],
                    'market_price'
                ].iloc[0] * point
                sl_amt = -fut_pnl
                net = entry['opt_in'] - exit_opt - sl_amt
                records.append({'net': net, 'win': net > 0})
                position_active = False
                entry = {}
                continue

            # 到期
            if today >= entry['end_date']:
                records.append({'net': entry['opt_in'], 'win': True})
                position_active = False
                entry = {}
                continue

        # 空仓：信号 + 选合约
        if (not position_active
            and today in df_sig.index
            and df_sig.at[today, 'can_enter']):
            today_opt = today_opt.copy()
            today_opt['theo_ratio'] = (
                today_opt['market_price'] / today_opt['theo_price']
            )
            cands = today_opt[
                (today_opt['theo_ratio'] >= theo_ratio) &
                (today_opt['strike'] > today_opt['underlying']) &
                ((today_opt['contract_month'] - today_opt['date']).dt.days <= 40)
            ]
            if not cands.empty:
                sel = cands.sort_values('market_price', ascending=False).iloc[0]
                entry = {
                    'date':      today,
                    'price':     df_sig.at[today, 'front_settle'],
                    'opt_in':    sel['market_price'] * point,
                    'strike':    sel['strike'],
                    'end_date':  sel['contract_month']
                }
                position_active = True

    df = pd.DataFrame(records)
    return {
        'net_profit': df['net'].sum(),
        'win_rate':   df['win'].mean() if len(df) else np.nan,
        'trades':     len(df)
    }
# ---------- 2. 指标与信号 ----------
def compute_indicators(df, mom_long, mom_short, atr_short=14, atr_long=60):
    df = df.copy().sort_values('date')
    df['Mom_long']  = df['front_settle'].pct_change(mom_long)
    df['Mom_short'] = df['front_settle'].pct_change(mom_short)
    high, low, prev = df['front_high'], df['front_low'], df['front_settle'].shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev).abs(),
        (low - prev).abs()
    ], axis=1).max(axis=1)
    df['ATR_short'] = tr.rolling(atr_short).mean()
    df['ATR_long']  = tr.rolling(atr_long).mean()
    return df.dropna(subset=['Mom_long','Mom_short','ATR_short','ATR_long'])

def generate_signals(df_ind, short_thresh, atr_ratio):
    df = df_ind.copy()
    cond_long  = df['Mom_long']  < 0.0
    cond_short = df['Mom_short'] < short_thresh
    cond_vol   = df['ATR_short'] < df['ATR_long'] * atr_ratio

    df['can_enter'] = cond_long & cond_short & cond_vol
    df['fail_reason'] = np.where(~cond_long,  'Long Mom > 0',
                          np.where(~cond_short,'Short Mom > thresh',
                          np.where(~cond_vol,   'ATR too high','')))
    return df

# 载入历史期权和期货数据
opt_df = pd.read_csv("NK225MWE_call_backtest_2023_fitted_2503updated.csv", parse_dates=["date","contract_month"])
df_raw = load_futures_excel("N225minif_2025.xlsx", "日中日足", "日付", "2023-06-01", "2025-06-26")

# 网格参数
mom_long_periods  = [8, 10]
mom_short_periods = [1, 2,3, 5]
short_threshs     = [-0.001,-0.005, -0.01]
atr_ratios        = [2.0, 2.5]

results = []
for mom_l, mom_s, thresh, atr_r in product(mom_long_periods, mom_short_periods, short_threshs, atr_ratios):
    df_ind = compute_indicators(df_raw, mom_long=mom_l, mom_short=mom_s)
    df_sig = generate_signals(df_ind, short_thresh=thresh, atr_ratio=atr_r)
    perf   = backtest(opt_df, df_sig)  # 复用你已有的 backtest()
    perf.update({
        'mom_long': mom_l,
        'mom_short': mom_s,
        'short_thresh': thresh,
        'atr_ratio': atr_r
    })
    results.append(perf)

df_res = pd.DataFrame(results).sort_values('net_profit', ascending=False).reset_index(drop=True)
print(df_res)


      net_profit  win_rate  trades  mom_long  mom_short  short_thresh  \
0   1.218986e+06  0.833333      36         8          1        -0.005   
1   1.218986e+06  0.833333      36         8          1        -0.005   
2   1.203069e+06  0.809524      42         8          1        -0.001   
3   1.203069e+06  0.809524      42         8          1        -0.001   
4   1.090836e+06  0.783784      37        10          1        -0.001   
5   1.090836e+06  0.783784      37        10          1        -0.001   
6   1.086208e+06  0.740000      50         8          3        -0.001   
7   1.086208e+06  0.740000      50         8          3        -0.001   
8   1.055583e+06  0.795918      49         8          2        -0.001   
9   1.055583e+06  0.795918      49         8          2        -0.001   
10  9.792245e+05  0.770833      48         8          5        -0.001   
11  9.792245e+05  0.770833      48         8          5        -0.001   
12  9.742283e+05  0.804348      46        10       

# 日经期货 Covered Call “期货腿”总体思路

下面以最通俗的方式介绍“期货多头＋卖出看涨期权”（Covered Call）套利中，**期货腿**的总体思路、关键术语和应对各种市场情况。最后附上一个简单的 ASCII 流程图，帮助你快速抓住核心。

---

## 一、总体思路

1. **策略目标**  
   - 通过“买入日经225期货多头”＋“同日卖出等 Delta 的看涨期权”，力图用期权端挣到的时间价值补偿期货腿的风险波动。  
   - 期货多头主要用来对冲期权的 Δ（Delta）敞口，理论上把大部分方向性风险中和，只赚取“隐含波动率 > 实际波动率”之间的差值。

2. **两大核心环节**  
   1. **择时买入期货（期货腿）**  
      - 只有在“趋势向上、短期动能强、波动适中、流动性充足”时才做多。  
      - 给自己留出一个“止损点”，一旦期货价格跌破该点就平仓（买回期权同时平掉期货），控制最大亏损。  
   2. **同日卖出看涨期权（期权腿）**  
      - 卖出一个月左右到期、轻度或适度虚值的看涨期权，赚取权利金。  
      - 每笔期权交易都要配合期货腿做 Δ 对冲（通常以当天的 Delta → 买入等量的期货），并在持仓期内动态监控 Delta、Gamma、Vega 等风险。

3. **为什么要这样做？**  
   - **期货腿**负责锁定方向性（如果市场上涨，期货赚钱；下跌则亏损）；  
   - **期权腿**负责赚取“卖权利金”（若波动较小、期权隐含波动高，就能赚到溢价）；  
   - 当隐含波动（IV）持续大于未来实际波动（RV）时，卖期权的收益会超过期货腿的亏损 → 整体组合可以正向获利。

---

## 二、关键术语 & 指标

以下术语会频繁出现，请先理解它们的含义和作用：

1. **MA（移动均线，Moving Average）**  
   - 计算方式：将过去 N 天的收盘价求平均。  
   - 作用：表示“过去 N 天收盘价的平滑曲线”，常见的 N=20（约一个月）。  
   - 交易意义：  
     - `收盘价 > MA20` → 中期趋势向上，可优先做多；  
     - `收盘价 < MA20` → 中期趋势偏弱，需谨慎或空仓。

2. **Mom（动量，Momentum）**  
   - 计算方式：过去 M 天的涨跌幅（`今日收盘 ÷ M 天前收盘 − 1`），常见 M=5（约一周）。  
   - 作用：表示“短期内价格的涨跌力度”，Mom5 > 0 说明过去一周整体上涨。  
   - 交易意义：  
     - `Mom5 > 0` 且 `收盘价 > MA20` → 趋势＋动能都偏多，做多信号更强；  
     - `Mom5 < 0` → 短期动能偏空，谨慎做多。

3. **ATR（平均真实波幅，Average True Range）**  
   - 计算方式：  
     1. 当日真实波幅 TR = max(当日最高−当日最低, 当日最高−昨收, 当日最低−昨收)；  
     2. 对过去 P 天的 TR 做简单平均，得到 `ATR_P`。  
   - 常用参数：  
     - `ATR14`（过去 14 天）称为“短期 ATR”，衡量 2 周内的平均波动；  
     - `ATR60`（过去 60 天）称为“长期 ATR”，衡量 3 个月内的平均波动。  
   - 交易意义：  
     - **ATR14** 用来 **设止损距离**：`止损价 = 入场价 − k × ATR14`，k 可调（如 1.0、1.2）。  
     - **ATR14 vs ATR60** 用作 **波动过滤**：  
       - 若 `ATR14 < ATR60 × atr_filter_ratio`（默认比例 = 1），说明“当前波动低于历史平均”，更适合在波动放大前进场。

4. **流动性（Volume / Open Interest）**  
   - **成交量 (Volume)**：当日买卖合约总量，衡量活跃度。通常要 `Volume ≥ vol_threshold`（如 2 万手）才能进场。  
   - **持仓量 (Open Interest)**：当日所有未平仓合约总量，也衡量“市场参与热度”。通常要 `Open Interest ≥ oi_threshold`（如 1 万手）以上。

5. **Delta / Gamma / Vega（期权风险指标）**  
   - **Delta**：期权价格对标的价格变动的敏感度。卖看涨期权后，需在期货腿多买入等量 Delta（如 Δ=0.3，则买 0.3 张期货）来对冲。  
   - **Gamma**：衡量 Delta 对标的价格二阶敏感度，即“Delta 会如何因价格变化而变化”。Gamma 大时，Delta 变化迅速，需要更频繁地对冲。  
   - **Vega**：衡量期权价格对隐含波动率变化的敏感度。一旦隐含波动（IV）暴涨，你卖空的期权会立刻亏损，期货对冲无法弥补这部分收支。

---

## 三、各类市场情况与对应策略

以下几种常见行情情况，以及我们在“期货腿”上对应的做法：

1. **趋势向上 + 动量强 + 波动适中（ATR14 < ATR60）+ 流动性好**  
   - **做法**：  
     1. 当日收盘满足：`收盘 > MA20` 且 `Mom5 > 0`，并且 `ATR14 < ATR60`、`Volume ≥ vol_threshold`、`OI ≥ oi_threshold` → 发出“入场信号(can_enter = True)”。  
     2. **收盘价看多、买入期货多头**，同时**卖出相应 Delta 的看涨期权**。  
     3. 设置止损价（默认为 `入场价 − 1×ATR14`），并挂限价止损单。  

2. **趋势向上但波动已经很大（ATR14 ≥ ATR60×ratio）**  
   - **做法**：  
     - 暂时**不入场**，因为市场波动已经偏大，风险不易控制。  
     - 等待短期波动回落（ATR14 再次下降到 ATR60 之下）或趋势更稳固时再考虑入场。  

3. **趋势向下（收盘 ≤ MA20）或短期动量为负（Mom5 ≤ 0）**  
   - **做法**：  
     - 不做多、不卖 Covered Call，可以考虑空一部分期货但需与空看跌期权或买入看跌期权对冲（非本策略范围）。  
     - “期货腿”此时保持空仓，等待趋势向上再介入。

4. **触发止损（收盘 ≤ 止损价）**  
   - **做法**：  
     1. 当日收盘价如果跌破 `止损价 = 入场价 − k×ATR14`（或百分比止损），立即**平掉期货多头 + 买回期权**。  
     2. 记录“止损平仓日”和“止损价”，结束本次持仓。  
     3. 之后需要等待满足新一轮“入场条件”时再做下一笔。

5. **隔夜或重要事件**  
   - 若不打算隔夜持仓：**收盘前必须平仓**，避免夜间跳空风险。  
   - 若要隔夜持仓：需留足保证金，并预先在止损价附近挂好“限价止损单”；一旦夜盘行情剧烈跳空，自动平仓减少损失。  

6. **隐含波动骤升（Vega 风险）**  
   - 若卖出期权后，突发利空消息导致 IV 暴涨，卖空期权价值会大幅上涨，期货对冲无法抵消这部分亏损：  
     - **做法**：一旦检测到“当天 IV 大涨 > 阈值”（比如暴涨 5%以上），就要**立即买回期权 + 平仓期货**，把风险止住。  

---

## 四、简易流程图（ASCII）

下面的流程图展示了“期货腿”从“检测入场到止损离场”这一段最关键的逻辑：



In [2]:
def fetch_continuous_futures_yq(symbol="NIY=F", start="2023-01-01", end="2025-06-09", max_retries=5):
    """
    使用 yahooquery 拉取日经225期货连续合约日线数据，带重试与退避。
    返回 DataFrame，索引为日期，列 ['open','high','low','close','volume'].
    """
    for attempt in range(1, max_retries+1):
        try:
            tkr = Ticker(symbol, asynchronous=True)
            df = tkr.history(start=start, end=end)
            df_sym = df.xs(symbol, level=0)
            break
        except Exception as e:
            if attempt == max_retries:
                raise
            backoff = (2 ** (attempt-1)) + random.random()
            print(f"尝试{attempt}失败：{e}，{backoff:.1f}s后重试")
            time.sleep(backoff)
    df_sym.index = pd.to_datetime(df_sym.index)
    df_sym = df_sym.sort_index()
    df_cont = pd.DataFrame({
        'front_open':   df_sym['open'],
        'front_high':   df_sym['high'],
        'front_low':    df_sym['low'],
        'front_settle': df_sym['close'],
        'front_volume': df_sym['volume'],
    }, index=df_sym.index)
    return df_cont

In [13]:
#生成期货端信号（原始版信号mom+atr）
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# 1. 读取期货数据，并按时间过滤
def load_futures_excel(
    excel_path="N225minif_2025.xlsx",
    sheet_name="日中日足",
    date_col="日付",
    start_date="2023-01-01",
    end_date=None
) -> pd.DataFrame:
    """
    从 Excel 加载期货数据，解析日期列，
    并根据 start_date 和 end_date 过滤数据。
    """
    df = pd.read_excel(
        excel_path,
        sheet_name=sheet_name,
        parse_dates=[date_col]
    )
    df = df.rename(columns={
        date_col: "date",
        "終値": "front_settle",
        "高値": "front_high",
        "安値": "front_low",
        "出来高": "front_volume"
    })
    df = df.sort_values("date").set_index("date")
    # 过滤日期
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date) if end_date else df.index.max()
    df = df.loc[(df.index >= start) & (df.index <= end)]
    return df

# 2. 计算技术指标
def compute_indicators(df, ma_window=20, mom_period=10, atr_short=14, atr_long=60):
    df = df.copy()
    df['MA'] = df['front_settle'].rolling(ma_window).mean()
    df['Mom'] = df['front_settle'].pct_change(mom_period)
    high, low, prev = df['front_high'], df['front_low'], df['front_settle'].shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev).abs(),
        (low - prev).abs()
    ], axis=1).max(axis=1)
    df['ATR_short'] = tr.rolling(atr_short).mean()
    df['ATR_long']  = tr.rolling(atr_long).mean()
    return df

# 3. 生成入场信号
def generate_signals(df, vol_threshold=1000, atr_ratio=2.0):
    df = df.copy()
    df['can_enter'] = (
        (df['front_settle'] > df['MA']) &
        (df['Mom'] > 0) &
        (df['ATR_short'] < df['ATR_long'] * atr_ratio) &
        (df['front_volume'] > vol_threshold)
    )
    return df

# 4. 主流程及交互式可视化
if __name__ == "__main__":
    # 可以修改 start_date 和 end_date
    df_raw = load_futures_excel(
        excel_path="N225minif_2025.xlsx",
        sheet_name="日中日足",
        date_col="日付",
        start_date="2023-03-01",
        end_date="2025-06-16"
    )
    df_ind = compute_indicators(df_raw)
    df_sig = generate_signals(df_ind)

    # 交互式图表
    fig = go.Figure()
    # 收盘价折线
    fig.add_trace(go.Scatter(
        x=df_sig.index, y=df_sig['front_settle'],
        mode='lines', name='收盘价', line=dict(color='blue')
    ))
    # 入场信号标记
    ent = df_sig[df_sig['can_enter']]
    fig.add_trace(go.Scatter(
        x=ent.index, y=ent['front_settle'],
        mode='markers', name='入场信号',
        marker=dict(symbol='triangle-up', size=10, color='green')
    ))

    fig.update_layout(
        title="期货收盘价与入场信号（交互式）",
        xaxis_title="日期",
        yaxis_title="价格",
        hovermode='x unified'
    )
    fig.show()


In [3]:
# 计算技术指标（动量波动版）
# 1. 读取期货数据，并按时间过滤
def load_futures_excel(
    excel_path="N225minif_2025.xlsx",
    sheet_name="日中日足",
    date_col="日付",
    start_date="2023-01-01",
    end_date=None
) -> pd.DataFrame:
    """
    从 Excel 加载期货数据，解析日期列，
    并根据 start_date 和 end_date 过滤数据。
    """
    df = pd.read_excel(
        excel_path,
        sheet_name=sheet_name,
        parse_dates=[date_col]
    )
    df = df.rename(columns={
        date_col: "date",
        "終値": "front_settle",
        "高値": "front_high",
        "安値": "front_low",
        "出来高": "front_volume"
    })
    df = df.sort_values("date").set_index("date")
    # 过滤日期
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date) if end_date else df.index.max()
    df = df.loc[(df.index >= start) & (df.index <= end)]
    return df
# 2. 计算技术指标（动量波动版，增加短期动量）
def compute_indicators(
    df,
    ma_short_window=20,
    ma_long_window=60,
    mom_period=8,
    mom_short_period=1,
    atr_short=10,
    atr_long=50
):
    df = df.copy()
    df['MA_short']    = df['front_settle'].rolling(ma_short_window).mean()
    df['MA_long']     = df['front_settle'].rolling(ma_long_window).mean()
    # 主动量（8 天）
    df['Mom_long']    = df['front_settle'].pct_change(mom_period)
    # 短期动量（1 天）
    df['Mom_short']   = df['front_settle'].pct_change(mom_short_period)

    high, low, prev = df['front_high'], df['front_low'], df['front_settle'].shift(1)
    tr = pd.concat([
        (high - low).abs(),
        (high - prev).abs(),
        (low - prev).abs()
    ], axis=1).max(axis=1)
    df['ATR_short']   = tr.rolling(atr_short).mean()
    df['ATR_long']    = tr.rolling(atr_long).mean()
    return df

# 3. 生成入场信号，并标注失败原因（加上短期动量阈值）
def generate_signals(
    df_ind: pd.DataFrame,
    mom_down_threshold: float = 0.0,
    mom_short_threshold: float = -0.005,
    atr_ratio: float = 2.0
) -> pd.DataFrame:
    """
    fail_reason:
      - “强下跌”      : 长期动量 <= mom_down_threshold
      - “急跌确认”    : 短期动量 <= mom_short_threshold
      - “超高波动”    : ATR_short/ATR_long >= atr_ratio
      - 否则为空串 -> can_enter=True
    """
    df = df_ind.copy()
    reasons = []

    for _, row in df.iterrows():
        if row['Mom_long'] <= mom_down_threshold:
            reasons.append('强下跌')
        elif row['Mom_short'] <= mom_short_threshold:
            reasons.append('急跌确认')
        elif row['ATR_short'] >= row['ATR_long'] * atr_ratio:
            reasons.append('超高波动')
        else:
            reasons.append('')

    df['fail_reason'] = reasons
    df['can_enter']   = df['fail_reason'] == ''
    return df

In [4]:
df_raw = load_futures_excel(
        excel_path="N225minif_2025.xlsx",
        sheet_name="日中日足",
        date_col="日付",
        start_date="2023-06-01",
        end_date="2025-06-27"
    )
df_ind = compute_indicators(df_raw)
df_sig = generate_signals(df_ind)

# 交互式图表
fig = go.Figure()

# 1) 收盘价
fig.add_trace(go.Scatter(
    x=df_sig.index,
    y=df_sig['front_settle'],
    mode='lines',
    name='收盘价',
    line=dict(color='blue')
))

# 2) 可入场点
ent = df_sig[df_sig['can_enter']]
fig.add_trace(go.Scatter(
    x=ent.index,
    y=ent['front_settle'],
    mode='markers',
    name='入场信号',
    marker=dict(symbol='triangle-up', size=10, color='green')
))

# 3) 因“强下跌”被禁止入场的点
down = df_sig[df_sig['fail_reason']=='强下跌']
fig.add_trace(go.Scatter(
    x=down.index,
    y=down['front_settle'],
    mode='markers',
    name='禁止入场：强下跌',
    marker=dict(symbol='x', size=8, color='red')
))

# 4) 因“急跌确认”被禁止入场的点（短期动量跌破阈值）
acute = df_sig[df_sig['fail_reason']=='急跌确认']
fig.add_trace(go.Scatter(
    x=acute.index,
    y=acute['front_settle'],
    mode='markers',
    name='禁止入场：急跌确认',
    marker=dict(symbol='triangle-down', size=8, color='orange')
))

# 5) 因“超高波动”被禁止入场的点
vol = df_sig[df_sig['fail_reason']=='超高波动']
fig.add_trace(go.Scatter(
    x=vol.index,
    y=vol['front_settle'],
    mode='markers',
    name='禁止入场：超高波动',
    marker=dict(symbol='x', size=8, color='purple')
))

fig.update_layout(
    title="期货收盘价与入场/禁止入场标记",
    xaxis_title="日期",
    yaxis_title="价格",
    hovermode='x unified'
)

fig.show()

In [14]:
df_sig.iloc[40:80]

,始値,front_high,front_low,front_settle,front_volume,MA_short,MA_long,Mom,ATR_short,ATR_long,fail_reason,can_enter
date,,,,,,,,,,,,
2023-07-27,32490,32940,32430,32920,471166,32642.50,NaN,0.015579,472.500000,NaN,,True
2023-07-28,32490,32835,32000,32765,1124913,32621.25,NaN,0.015025,501.428571,NaN,,True
2023-07-31,33095,33390,32985,33260,737199,32601.75,NaN,0.019933,516.785714,NaN,,True
2023-08-01,33280,33470,33190,33405,461123,32605.75,NaN,0.023438,493.571429,NaN,,True
2023-08-02,33075,33130,32585,32610,745311,32570.25,NaN,0.000307,507.500000,NaN,,True
2023-08-03,32340,32430,32085,32180,702377,32542.75,NaN,-0.022479,498.571429,NaN,强下跌,False
2023-08-04,31940,32280,31900,32145,512013,32528.50,NaN,-0.018923,512.500000,NaN,强下跌,False
2023-08-07,31890,32320,31790,32320,469825,32537.50,NaN,-0.028262,514.642857,NaN,强下跌,False
2023-08-08,32395,32525,32215,32290,458773,32540.00,NaN,-0.033378,493.571429,NaN,强下跌,False


In [ ]:
 #生成入场信号（加入双均线条件）
def generate_signals(
    df_ind,
    mom_down_threshold: float = 0.0,
    atr_ratio: float = 1.2
) -> pd.DataFrame:
    """
    只在以下两种“风险期”禁止建仓：
      1) Mom ≤ mom_down_threshold  → 强下跌
      2) ATR_short/ATR_long ≥ atr_ratio → 超高波动
    同时要求短期均线在长期均线之上，否则也禁止建仓。
    其余时间 can_enter=True。
    """
    df = df_ind.copy()
    reasons = []

    for _, row in df.iterrows():
        if row['Mom'] <= mom_down_threshold:
            reasons.append('强下跌')
        elif row['ATR_short'] >= row['ATR_long'] * atr_ratio:
            reasons.append('超高波动')
        elif row['MA_short'] <= row['MA_long']:
            reasons.append('弱势或横盘')
        else:
            reasons.append('')

    df['fail_reason'] = reasons
    df['can_enter']   = df['fail_reason'] == ''
    return df

In [ ]:
def build_df_cont_from_yf(df_yf: pd.DataFrame) -> pd.DataFrame:

    """
    输入：yfinance 拉取到的 DataFrame df_yf，索引为 MultiIndex(symbol, date)，
          列 ['open','high','low','close','volume','adjclose']。
    输出：单层索引的 df_cont，日期为索引，列为：
          ['front_open','front_high','front_low','front_settle','front_volume']
          
    """
    # 1) 如果 index 是 MultiIndex，先取出 symbol="NIY=F" 的部分；否则直接当 df_yf 用
    if isinstance(df_yf.index, pd.MultiIndex):
        # 假设第一层 index 名称是 'symbol'
        # 有两种常见写法：.xs 或者 .loc
        # 我们这里用 .xs 来提取第一层为 "NIY=F" 的子表
        df_sym = df_yf.xs("NIY=F", level=0).copy()
    else:
        # 如果没有多重索引，就直接把 df_yf 当作目标（但本例里应该是 MultiIndex）
        df_sym = df_yf.copy()

    # 2) 确保索引是 DatetimeIndex（如果被当成字符串），并按日期排序
    if not pd.api.types.is_datetime64_any_dtype(df_sym.index):
        df_sym.index = pd.to_datetime(df_sym.index)
    df_sym = df_sym.sort_index()

    # 3) 把需要的列重命名为前面例子中用的列名：
    #    yfinance 的 'open','high','low','close','volume' → 对应 'front_open','front_high',…
    df_cont = pd.DataFrame({
        'front_open':   df_sym['open'],
        'front_high':   df_sym['high'],
        'front_low':    df_sym['low'],
        'front_settle': df_sym['close'],
        'front_volume': df_sym['volume'],
        # 这里 yfinance 没有持仓量（Open Interest），所以先填 NaN 或 0，后续若有券商接口可补充
    }, index=df_sym.index)

    return df_cont
# 之前示例里定义的两个函数（不必再改动）：
def compute_futures_indicators(df_cont: pd.DataFrame) -> pd.DataFrame:
    df = df_cont.copy()
    # 1) MA20
    df['MA20'] = df['front_settle'].rolling(window=20, min_periods=20).mean()
    # 2) 5 日动量
    df['Mom5'] = df['front_settle'].pct_change(periods=5)
    # 3) True Range 及 ATR
    high = df['front_high']
    low  = df['front_low']
    prev_close = df['front_settle'].shift(1)
    tr1 = (high - low).abs()
    tr2 = (high - prev_close).abs()
    tr3 = (low  - prev_close).abs()
    df['TR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df['ATR14'] = df['TR'].rolling(window=14, min_periods=14).mean()
    df['ATR60'] = df['TR'].rolling(window=60, min_periods=60).mean()
    return df
def generate_entry_stop_signals(df_ind: pd.DataFrame,
                                volume_threshold: float = 20000,
                                atr_multiplier: float = 1.0) -> pd.DataFrame:
    df = df_ind.copy()
    # 1) 生成 can_enter
    df['can_enter'] = False
    df.loc[
        (df['front_settle'] > df['MA20']) &
        (df['Mom5'] > 0) &
        (df['ATR14'] < df['ATR60']) &
        (df['front_volume'] > volume_threshold),
        'can_enter'
    ] = True

    # 2) 初始化 entry_price 与 stop_loss_price
    df['entry_price'] = np.nan
    df['stop_loss_price'] = np.nan

    in_position = False
    entry_atr14 = None

    # 3) 遍历每个交易日，做开仓/止损判断
    for idx, row in df.iterrows():
        if not in_position:
            # 如果当前未持仓，且当天 can_enter 为 True，就记为开仓
            if row['can_enter']:
                in_position = True
                entry_price = row['front_settle']  # 也可改成 row['front_open']
                entry_atr14 = row['ATR14']
                df.at[idx, 'entry_price'] = entry_price
                df.at[idx, 'stop_loss_price'] = entry_price - atr_multiplier * entry_atr14
            else:
                continue
        else:
            # 如果已经持仓，要看是否触发止损
            last_sl = df.loc[:idx, 'stop_loss_price'].dropna().iloc[-1]
            if row['front_settle'] <= last_sl:
                # 触发止损，结束持仓
                in_position = False
                entry_atr14 = None
                # 注意：当日止损后不立刻再开新仓，除非下一行里 can_enter 又变 True
            else:
                continue

    return df
# ============================================================
# —— 整体运行示例 —— （假设 df_yf 已经由 yfinance 拉好）
# ============================================================

# 例如你通过 yfinance 得到 df_yf：
#

#tk = Ticker("NIY=F")
#df_all = tk.history(start="2022-06-01", end="2025-06-01", interval="1d")
# 1) 把 yfinance 的 df_yf 转成 df_cont
df_cont = build_df_cont_from_yf(df_all)

# 2) 在 df_cont 基础上计算技术指标
df_ind = compute_futures_indicators(df_cont)

# 3) 生成入场/止损信号
#    这里演示的阈值只是示例，你可以根据日经期货的日均“成交量”和“持仓量”自行微调
df_signals = generate_entry_stop_signals(df_ind,
                                         volume_threshold=20000,  # 例如最低成交量 2 万    
                                         atr_multiplier=1.0)

# 4) 把 entry_price 和 stop_loss_price 向下填充，便于在持仓期随时读取入场与止损价
df_signals['entry_price_ffill'] = df_signals['entry_price'].ffill()
df_signals['stop_loss_price_ffill'] = df_signals['stop_loss_price'].ffill()

# 5) 简单看一下几行结果，验证入场/止损是否符合预期
cols_to_show = [
    'front_settle',
    'MA20', 'Mom5', 'ATR14', 'ATR60',
    'can_enter', 'entry_price', 'stop_loss_price',
    'entry_price_ffill', 'stop_loss_price_ffill'
]
print(df_signals[cols_to_show].dropna(subset=['MA20']).tail(20))


            front_settle      MA20      Mom5       ATR14       ATR60  \
date                                                                   
2025-05-02       37210.0  34659.95  0.034329  412.500000  592.817643   
2025-05-05       36925.0  34897.70  0.027550  385.714286  597.317643   
2025-05-06       36430.0  35112.20  0.015895  405.000000  595.484310   
2025-05-07       36925.0  35346.45  0.021580  395.357143  594.234310   
2025-05-08       37319.0  35472.65  0.013277  379.571429  598.217643   
2025-05-09       37470.0  35669.20  0.006987  384.571429  603.884310   
2025-05-12       38760.0  35927.95  0.049695  433.857143  621.266667   
2025-05-13       38370.0  36134.20  0.053253  448.857143  623.850000   
2025-05-14       37720.0  36296.70  0.021530  488.857143  628.683333   
2025-05-15       37755.0  36492.45  0.011683  478.857143  623.683333   
2025-05-16       37995.0  36669.45  0.014011  512.785714  622.350000   
2025-05-19       37605.0  36843.45 -0.029799  561.000000  621.26

In [ ]:
# --- Module: Fetch Futures Data via Selenium ---
def fetch_jpx_future_quotes(disptype: str, targets=None) -> pd.DataFrame:
    """
    抓取并过滤 JPX 先物价格信息（迷你日経225及标准日経225）。
    disptype: 'day_through'（日通し）、'daytime'（日中）、'night'（夜間）
    targets: list of contract_type to keep, e.g. ['日経225先物', '日経225mini']
    返回包含以下字段的 DataFrame:
      ['contract_type','contract_month','trade_date',
       'open_price','high_price','low_price','last_price',
       'change','volume_value','ask_price','ask_size',
       'bid_price','bid_size','settlement_price',
       'limit_upper','limit_lower','open_interest','session']
    """
    url = f"https://port.jpx.co.jp/jpxhp/main/index.aspx?f=future&disptype={disptype}"

    # 启动无头浏览器
    opts = Options()
    opts.add_argument("--headless")
    driver = webdriver.Chrome(options=opts)
    driver.get(url)
    driver.implicitly_wait(5)

    # 获取渲染后的表格 HTML
    table = driver.find_element("css selector", "table.tbl-s2")
    html = table.get_attribute("outerHTML")
    driver.quit()

    # 用 pandas 解析 HTML
    df = pd.read_html(html)[0]
    df.columns = [
        'contract_type', 'contract_month', 'trade_date',
        'open_price', 'high_price', 'low_price', 'last_price',
        'change', 'volume_value', 'ask_price', 'ask_size',
        'bid_price', 'bid_size', 'settlement_price', 'limit_range', 'open_interest'
    ]

    # 只保留目标合约
    if targets is not None:
        df = df[df['contract_type'].isin(targets)].reset_index(drop=True)

    # 按空白字符安全拆分限价范围，如 "40,990  34,930"
    limits = df['limit_range'].astype(str).str.split(r'\s+', expand=True)
    df['limit_upper'] = limits[0].str.replace(',', '').astype(float)
    df['limit_lower'] = limits[1].str.replace(',', '').astype(float)
    df = df.drop(columns=['limit_range'])

    # 转换日期
    df['trade_date'] = pd.to_datetime(df['trade_date'], format='%m/%d').apply(
        lambda dt: dt.replace(year=pd.Timestamp.now().year)
    )

    # 转换数值列：先提取数字部分，再转换
    num_cols = [
        'open_price','high_price','low_price','last_price',
        'change','volume_value','ask_price','ask_size',
        'bid_price','bid_size','settlement_price',
        'limit_upper','limit_lower','open_interest'
    ]
    for col in num_cols:
        # 提取字符串中的首个数字或带千分位的部分
        # 例如 "38,000 (05/30) (17:00)" -> "38,000"
        num_str = df[col].astype(str).str.extract(r'([\d,]+)')[0]
        df[col] = pd.to_numeric(num_str.str.replace(',', ''), errors='coerce')

    # 标记时段
    df['session'] = disptype
    return df

# --- Example Usage: Fetch and Combine Sessions ---
sessions = ['day_through', 'daytime', 'night']
targets = ['日経225先物', '日経225mini']
df_all = pd.concat([fetch_jpx_future_quotes(s, targets) for s in sessions], ignore_index=True)
print(df_all.head())


  contract_type contract_month trade_date  open_price  high_price  low_price  \
0       日経225先物         25年6月限 2025-06-02       38000       38040      37390   
1       日経225先物         25年9月限 2025-06-02       37990       38000      37390   
2       日経225先物        25年12月限 2025-06-02       37720       37720      37720   
3     日経225mini         25年6月限 2025-06-02       38000       38045      37390   
4     日経225mini         25年7月限 2025-06-02       37970       38015      37375   

   last_price  change  volume_value  ask_price  ask_size  bid_price  bid_size  \
0       37680     280         13221      37710         2    37640.0       3.0   
1       37660     250           228      38120         1    37260.0       2.0   
2       37720      60             1      39400         1        NaN       NaN   
3       37680     280        268554      37690         2    37655.0       2.0   
4       37650     320          3136      37840         1    37400.0       1.0   

   settlement_price  open_intere